# Data Cleaning

## Import Packages

In [13]:
import pandas as pd
import numpy as np
from itertools import chain

## Get Country Codes

In [15]:
country_df = pd.read_excel(
    '../data/unwto-all-data-download_2022.xlsx',
    sheet_name = 'Inbound Tourism-Arrivals',
    header = 2
)

In [16]:
country_df = country_df[['C.','Basic data and indicators']][
    (country_df['Basic data and indicators'].isna() == False) & 
    (country_df['C.'].isna() == False)
]

In [21]:
countries = country_df.rename(columns = {
    'Basic data and indicators':'country'
})

## Function to Standardize Data Cleaning

In [28]:
def cleanse_tourism_df(sheet):
    
    # read excel sheet
    df = pd.read_excel(
        '../data/unwto-all-data-download_2022.xlsx',
        sheet_name = sheet,
        header = 2
    )
    
    # remove irrelevant rows
    df = df[
        (df['S.'].isna() == False) & 
        (df['S.'] != 0)
    ]
    
    # utilizing the attribute number column with table name, as numbers are repeating across tables
    df['table_measure'] = sheet.replace(' ','_').replace('-','_').lower() + df['S.'].map('{:,.2f}'.format)
    
    # defining the columns to keep
    col_list = [['C.','table_measure'], list(range(1995,2023))]
    col_list = list(chain(*col_list))
    df = df[col_list]
    
    # utilizing pd.melt() to put the years onto rows
    df = pd.melt(df,
                 id_vars=['C.', 'table_measure'], 
                 var_name='year',
                 value_name='value')
    
    # replacing '..' values with NaN
    df['value'] = df['value'].replace('..',np.nan)
    
    # pivoting to put the attributes onto columns
    df = df.pivot(index=['C.', 'year'], columns='table_measure', values='value').reset_index()
    
    return df

In [29]:
# inbound tourism tables
inbound_accommodations = cleanse_tourism_df('Inbound Tourism-Accommodation')
inbound_regions = cleanse_tourism_df('Inbound Tourism-Regions')
inbound_arrivals = cleanse_tourism_df('Inbound Tourism-Arrivals')
inbound_purpose = cleanse_tourism_df('Inbound Tourism-Purpose')
inbound_transport = cleanse_tourism_df('Inbound Tourism-Transport')
inbound_expenditure = cleanse_tourism_df('Inbound Tourism-Expenditure')

# domestic tourism tables
domestic_trips = cleanse_tourism_df('Domestic Tourism-Trips')
domestic_accommodations = cleanse_tourism_df('Domestic Tourism-Accommodation')

# outbound tourism tables
outbound_departures = cleanse_tourism_df('Outbound Tourism-Departures')
outbound_expenditure = cleanse_tourism_df('Outbound Tourism-Expenditure')

# tourism industry table
tourism_industry = cleanse_tourism_df('Tourism Industries')

# tourism employment table
employment = cleanse_tourism_df('Employment')